In [1]:
#%reset
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
sys.path.append('./modules')

import warnings
#import re

import pandas as pd
#import numpy as np
from sklearn import metrics

## import modules to build pipelines
import pipemodules as pm
import projecthandle as proj
import run_grid as rg

#% matplotlib inline

In [2]:
import pickle
import pipemodules as pm
import pandas as pd
import numpy as np


def quality_filter(all_data, min_train_score=0.75, max_diff=0.15):

    results = all_data.eval_results

    results.reset_index(drop=True, inplace=True)

    ## filter results and eliminate poor models
    for i in range(0,len(results)):
        if results.mean_train_score[i] > min_train_score \
        and abs(results.mean_test_score[i] - results.mean_train_score[i]) < max_diff:
            continue
        else: 
            results.drop(i, axis=0, inplace=True)

    results.reset_index(drop=True, inplace=True)

    ## create analysis set
    # set arrays for results
    dev_set_score = []
    eval_set_score = []
    dev_evs = []
    eval_evs = []
    dev_mae = []
    eval_mae = []
    dev_mse = []
    eval_mse = []
    dev_medae = []
    eval_medae = []
    method_ids = []
    parameters = []


    for i in range(0,len(results)):
        ## take method_ids and build estimator for current method


        # add calculated metrics, methods, and parameters to lists for results
        dev_set_score.append(clf.score(X_dev_temp, all_data.dev_set.y_raw))
        eval_set_score.append(clf.score(X_eval_temp, all_data.eval_set.y_raw))
        dev_evs.append(metrics.explained_variance_score(dev_predict, all_data.dev_set.y_raw))
        eval_evs.append(metrics.explained_variance_score(eval_predict, all_data.eval_set.y_raw))
        dev_mae.append(metrics.mean_absolute_error(dev_predict, all_data.dev_set.y_raw))
        eval_mae.append(metrics.mean_absolute_error(eval_predict, all_data.eval_set.y_raw))
        dev_mse.append(metrics.mean_squared_error(dev_predict, all_data.dev_set.y_raw))
        eval_mse.append(metrics.mean_squared_error(eval_predict, all_data.eval_set.y_raw))
        dev_medae.append(metrics.median_absolute_error(dev_predict, all_data.dev_set.y_raw))
        eval_medae.append(metrics.median_absolute_error(eval_predict, all_data.eval_set.y_raw))
        method_ids.append(string)
        parameters.append(params)

    # create dictionary object from results
    evaluation_results = {'dev_set_score':dev_set_score, 'eval_set_score':eval_set_score, \
                         'method_ids':method_ids, 'parameters':parameters, 'dev_evs':dev_evs, \
                         'eval_evs':eval_evs, 'dev_mae':dev_mae, 'eval_mae':eval_mae, \
                         'dev_mse': dev_mse, 'eval_mse':eval_mse, 'dev_median_ae':dev_medae, \
                         'eval_median_ae':eval_medae}
    

    # re-rank and sort filtered methods by test-score (r**2)
    analysis_set = pd.DataFrame(evaluation_results)
    array = np.array(analysis_set['eval_set_score'])
    temp = array.argsort()[::-1]
    ranks = np.empty(len(array), int)
    ranks[temp] = np.arange(len(array))
    analysis_set['rank_test_score'] = ranks
    analysis_set.sort_values(by='rank_test_score', inplace=True)
    analysis_set.reset_index(drop=True, inplace=True)
    
    return analysis_set

In [2]:
X,y,labels = proj.set_input('./input_files/hfe_descriptors_2.csv')
test = pm.preprocess()
test.data_split(X,y,labels)
results = rg.auto_grid(X, y, labels)
#proj.save_eval('./results',results)
#analysis_set = quality_filter(results)


#save_eval('./results',results)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)
/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/coordinate_descent.py:470: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)
/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/res3/Library/Python/2.7/lib/python/site-packages/sklearn/model_selection/_validation.py:238: UserWarning: With alpha=0, this

In [55]:
to_drop = []
for i in range(0, len(evaluation_results)-1):
    if str(evaluation_results.method_ids[i])==str(evaluation_results.method_ids[i+1]):
        to_drop.append(i+1)
    else:
        continue

for i in to_drop:
    evaluation_results.drop(i, inplace=True)

evaluation_results.sort_values(by='rank_test_score', inplace=True)
evaluation_results.reset_index(drop=True, inplace=True)

In [9]:
def save_eval(filename, all_data):
    devobj = proj.input_object()
    evalobj = proj.input_object()
    methobj = proj.method_object()
    projectobj = proj.project()
    devobj.create_object(all_data.X_train, all_data.Y_train, all_data.descriptor_matrix_train)
    evalobj.create_object(all_data.X_test, all_data.Y_test, all_data.descriptor_matrix_test)
    methobj.create_object(all_data.k_vals, all_data.selection_labels, all_data.ind_values, all_data.options)
    projectobj.save_project(all_data.results, devobj, evalobj, methobj, filename)

In [ ]:
    currind = 0
    to_drop = []
    for i in range(0, len(evaluation_results)-1):
        if abs(evaluation_results.eval_set_score[currind] - evaluation_results.eval_set_score[i+1]) < 0.01 \
            and abs(evaluation_results.dev_set_score[i] - evaluation_results.dev_set_score[i+1]) < 0.01:
            to_drop.append(i+1)
        else:
            currind = i
    
    for i in to_drop:
        evaluation_results.drop(i, inplace=True)
    
    evaluation_results.sort_values(by='rank_test_score', inplace=True)
    evaluation_results.reset_index(drop=True, inplace=True)
    
    to_drop = []
    for i in range(0, len(evaluation_results)-1):
        if str(evaluation_results.method_ids[i])==str(evaluation_results.method_ids[i+1]):
            to_drop.append(i+1)
        else:
            continue

    for i in to_drop:
        evaluation_results.drop(i, inplace=True)

    evaluation_results.sort_values(by='rank_test_score', inplace=True)
    evaluation_results.reset_index(drop=True, inplace=True)

In [2]:
test = proj.file_loader()

In [3]:
test.load_file('./results_files/results.p')

5


In [18]:
string = str(test.analysis_results.__class__)


In [21]:
test.X_test

[array([  7.72754700e-03,   0.00000000e+00,   9.80840000e+01,
          5.31074263e+00,   3.30473504e-01,   5.31074263e+00,
         -3.30473504e-01,   1.15136100e+02,   1.15220000e+02,
          5.00000000e+01,   8.64795918e-01,   8.64795918e-01,
         -7.72754700e-03,   2.53006046e+00,   2.93680656e+01,
          6.24264069e+00,   5.81999096e+00,   5.81999096e+00,
          3.91421356e+00,   3.61535507e+00,   3.61535507e+00,
          2.20288870e+00,   2.20288870e+00,   1.30767754e+00,
          1.30767754e+00,   7.47890958e-01,   7.47890958e-01,
         -4.00000000e-02,   6.14996123e+01,   7.96000000e+00,
          6.96000000e+00,   5.96000000e+00,   5.22687944e+01,
          5.73366748e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   3.26070237e+01,   1.29655780e+01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
        

In [11]:
len(test.X_train)

0

In [7]:
test.X_train

[]

In [12]:
len(np.array(a)[1:])

69

In [5]:
y

0     -1.91
1     -1.36
2     -6.69
3     -9.94
4     -2.94
5     -3.13
6     -0.90
7      1.09
8     -4.59
9     -9.51
10    -2.21
11    -5.90
12    -7.03
13    -5.06
14     1.23
15    -1.31
16    -2.49
17    -9.28
18     2.10
19    -5.72
20    -5.48
21    -4.72
22    -9.61
23     2.67
24    -4.84
25    -4.91
26    -0.77
27    -6.21
28    -7.40
29    -7.17
      ...  
40    -8.11
41    -2.40
42   -10.64
43    -9.30
44    -4.02
45     2.88
46     2.30
47    -8.83
48    -6.60
49    -4.52
50    -6.62
51    -4.50
52    -4.39
53    -4.09
54    -3.95
55    -3.79
56     0.75
57    -3.88
58    -5.46
59     3.13
60     3.16
61    -2.87
62    -0.90
63    -0.22
64    -5.10
65    -5.51
66    -2.68
67     2.11
68    -4.63
69    -4.91
Name: HFE, dtype: float64

In [3]:
dir(test)

['X_test',
 'X_train',
 'Y_test',
 'Y_train',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'data_split',
 'descriptor_matrix_test',
 'descriptor_matrix_train']